In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [11]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)

In [12]:
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That would be **Neil Armstrong**, an American astronaut. He took his famous "one small step" on July 20, 1969, during the Apollo 11 mission. 



In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://technopark.org/job-details/15243?job=Python%20Developer%20(%20For%20Night%20Shifts)")
page_data = loader.load().pop().page_content
print(page_data)


















 
















Technopark Job posting for Python Developer ( For Night Shifts)

MenuThis Is TechnoparkCompanies & JobsInfrastructure & Services< All Jobs REIZEND PRIVATE LIMITEDSBC Module 15, 
-2 Floor, Thejaswini Building, 
Technopark phase 1, Trivandrum , 695581http://www.reizendretail.inPython Developer ( For Night Shifts)Closing Date:08,Nov 2024Job Published: 24,Oct 2024Contact Email:  careers@reizendretail.in Brief DescriptionLocation: Trivandrum
Required Skills
•           Proficiency in Python programming with a strong grasp of core concepts.
•           Basic Data Structures and Algorithms: Familiarity with common data structures (lists, dictionaries, tuples) and basic algorithms (sorting, searching) is advantageous.
•           Problem-Solving Skills: Ability to analyse and solve programming problems using Python.
•           Understanding of Object-Oriented Programming (OOP): Basic understanding of OOP principles like classes, objects, inheritance, and enc

In [17]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res.content

'```json\n[\n  {\n    "role": "Python Developer ( For Night Shifts)",\n    "experience": "0 to 2 yrs. (Fresher’s can apply)",\n    "skills": [\n      "Proficiency in Python programming with a strong grasp of core concepts.",\n      "Basic Data Structures and Algorithms: Familiarity with common data structures (lists, dictionaries, tuples) and basic algorithms (sorting, searching) is advantageous.",\n      "Problem-Solving Skills: Ability to analyse and solve programming problems using Python.",\n      "Understanding of Object-Oriented Programming (OOP): Basic understanding of OOP principles like classes, objects, inheritance, and encapsulation",\n      "Basic knowledge in Data Science including but not limited to ETL, EDA, Data Processing, Visualization etc.",\n      "Familiarity with machine learning and deep learning models (e.g., Tensor Flow, Keras, PyTorch).",\n      "Experience working with SQL DMBS like PostgreSQL is a plus.",\n      "Basic understanding of Natural Language Proce

In [18]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Python Developer ( For Night Shifts)',
  'experience': '0 to 2 yrs. (Fresher’s can apply)',
  'skills': ['Proficiency in Python programming with a strong grasp of core concepts.',
   'Basic Data Structures and Algorithms: Familiarity with common data structures (lists, dictionaries, tuples) and basic algorithms (sorting, searching) is advantageous.',
   'Problem-Solving Skills: Ability to analyse and solve programming problems using Python.',
   'Understanding of Object-Oriented Programming (OOP): Basic understanding of OOP principles like classes, objects, inheritance, and encapsulation',
   'Basic knowledge in Data Science including but not limited to ETL, EDA, Data Processing, Visualization etc.',
   'Familiarity with machine learning and deep learning models (e.g., Tensor Flow, Keras, PyTorch).',
   'Experience working with SQL DMBS like PostgreSQL is a plus.',
   'Basic understanding of Natural Language Processing (NLP) concepts',
   'Knowledge of Python frameworks like

In [19]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\AMALA SALI\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:30<00:00, 2.76MiB/s]


In [27]:
links = collection.query(query_texts=["experience in python","experience in react"],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [31]:
job = json_res
skills = job[0]['skills']
skills

['Proficiency in Python programming with a strong grasp of core concepts.',
 'Basic Data Structures and Algorithms: Familiarity with common data structures (lists, dictionaries, tuples) and basic algorithms (sorting, searching) is advantageous.',
 'Problem-Solving Skills: Ability to analyse and solve programming problems using Python.',
 'Understanding of Object-Oriented Programming (OOP): Basic understanding of OOP principles like classes, objects, inheritance, and encapsulation',
 'Basic knowledge in Data Science including but not limited to ETL, EDA, Data Processing, Visualization etc.',
 'Familiarity with machine learning and deep learning models (e.g., Tensor Flow, Keras, PyTorch).',
 'Experience working with SQL DMBS like PostgreSQL is a plus.',
 'Basic understanding of Natural Language Processing (NLP) concepts',
 'Knowledge of Python frameworks like Django',
 'Familiarity with Jira and Agile methodology will be an added advantage.',
 'Certification or experience in data analysi

In [33]:
links = collection.query(query_texts=["skills"],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [34]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject:  Streamlining Your Processes with AI-Powered Solutions

Dear [Hiring Manager Name],

I came across your job opening for a Python Developer (For Night Shifts) and believe AtliQ could be a valuable partner in finding the right talent for your team. 

At AtliQ, we specialize in providing AI and software consulting services that help businesses like yours optimize processes, reduce costs, and scale efficiently.  We have a proven track record of success in developing tailored Python-based solutions, leveraging the expertise of our skilled developers.

You can explore some examples of our Python development expertise here:

* [https://example.com/ml-python-portfolio](https://example.com/ml-python-portfolio)
* [https://example.com/python-portfolio](https://example.com/python-portfolio)

We understand the importance of finding candidates with a strong foundation in Python programming, data structures, algorithms, and machine learning concepts. Our recruitment process is designed to id